In [3]:
# Imports!
from qiskit import Aer, IBMQ
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import EfficientSU2, QAOAAnsatz
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.algorithms import VQE
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Coding with a VQE 
## The Knapsack Problem

### The knapsack problem is the classic combinatorial optimization problem:

* Imagine you have a knapsack and you are packing it for a trip, 
* You have a whole bunch of items that you can put into it, each have unique values and weights. 
* Ideally you would want to take everything, but that would weight too much to carry,
* Instead you take a combination of items that have the highest value while being under the max weight.

### More fomally:

Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

### The Knapsack Problem in the real world:

This kind of problem often arises in resource allocation where the decision makers have to choose from a set of non-divisible projects or tasks under a fixed budget or time constraint, respectively.

# Let's see it in code! Setting up the problem

In this part, we will define the knapsack problem. Qiskit has a built-in function called `Knapsack` that we will use to define the problem. 

We define a list of values, a list of weights, the maximum weight, and then put it all together into a variable called `knapsack_problem` using the Knapsack function.

In [10]:
# BLOCK 2 - Defining the knapsack problem. We define a list of values, a list of weights, the maximum weight, and then
# put it all together into a variable called `problem' using the Knapsack function.
values = [4, 2, 5, 10, 13, 2, 5, 6]   # list of the values of items
weights = [3, 1, 2, 6, 6, 3, 4, 7]   # list of the weights of items
max_weight = 10            # maximum weight capacity (knapsack capacity)

knapsack_problem = Knapsack(values = values, weights = weights, max_weight = max_weight) #putting it all together

Feel free to be creative here - enter your own combination of values, weights, and max_weight! For a first pass, we recommend keeping the number of individual values < 5 . You can try bigger problems later!

## Using VQE to solve the problem

Let's solve the problem using VQE! We discussed roughly how VQE works in the prior discussion, now we can see it in action.

Here we convert the knapsack problem to a form that a quantum computer can understand, and also find out the number of qubits required to solve the problem.

In [11]:
# BLOCK 3- Converting the knapsack problem to a quantum circuit, i.e., a colletion of quantum gates (or operators).
# Run this block to convert the knapsack problem to a quantum operator, and print the number of qubits used.
operator, offset = QuadraticProgramToQubo().convert(knapsack_problem.to_quadratic_program()).to_ising()
print("number of qubits =",operator.num_qubits)

number of qubits = 12


Next, we will set up VQE. We will specify the tunable circuit using `EfficientSU2`. YOU WILL NEED TO FILL CODE HERE!!!

We will also specify the classical optimizer we want to use. Remember that hybrid algorithms have a tunable quantum circuit, and a classical computer that tells it which parameters to use. This optimizer is the classical part of the hybrid algorithm.  We will use a popular classical optimization algorithm known as SPSA. You do not need to fill code here - we have filled it out for you.

We will use a built-in Qiskit function named `VQE` to put together all this information to solve the problem using the VQE algorithm. Again, we have completed this step for you.

In [9]:
# We define our tunable circuit for VQE. Here, we will choose EfficientSU2.
# We define which classical optimizer we want to use - here we will use one called SPSA
# We tell our code to use VQE with the tunable circuit, the optimizer, and the quantum instance

In [6]:
# BLOCK 4 - Setting up VQE.

# FILL CODE HERE TO DEFINE THE TUNABLE QUANTUM CIRCUIT. SPECIFY THE CORRECT NUMBER OF QUBITS, AND CHOOSE THE NUMBER OF REPS AND THE LEVEL OF ENTANGLEMENT.
tunable_circuit = EfficientSU2(num_qubits = operator.num_qubits, reps = 3, entanglement = 'full',insert_barriers=True ) 

optimizer = SPSA(maxiter=15) # Classical optimizer

method = VQE(ansatz = tunable_circuit, optimizer = optimizer, quantum_instance = Aer.get_backend('qasm_simulator')) # Using the VQE algorithm 

Let's visualize the circuit!

In [8]:
# Block 5 - Visualize your circuit by drawing it
tunable_circuit.decompose().draw()

┌──────────┐ ┌──────────┐ ░                                              »
q_0: ┤ Ry(θ[0]) ├─┤ Rz(θ[8]) ├─░───■────■────■─────────■─────────■────────────»
     ├──────────┤ ├──────────┤ ░ ┌─┴─┐  │    │         │         │            »
q_1: ┤ Ry(θ[1]) ├─┤ Rz(θ[9]) ├─░─┤ X ├──┼────┼────■────┼────■────┼────■───────»
     ├──────────┤┌┴──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │    │    │    │       »
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[10]) ├─░──────┤ X ├──┼──┤ X ├──┼────┼────┼────┼────■──»
     ├──────────┤├───────────┤ ░      └───┘┌─┴─┐└───┘  │  ┌─┴─┐  │    │  ┌─┴─┐»
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[11]) ├─░───────────┤ X ├───────┼──┤ X ├──┼────┼──┤ X ├»
     ├──────────┤├───────────┤ ░           └───┘     ┌─┴─┐└───┘  │  ┌─┴─┐└───┘»
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[12]) ├─░─────────────────────┤ X ├───────┼──┤ X ├─────»
     ├──────────┤├───────────┤ ░                     └───┘     ┌─┴─┐└───┘     »
q_5: ┤ Ry(θ[5]) ├┤ Rz(θ[13]) ├─░───────────────────────────────┤ X ├──────────»
     ├──────────┤├───────────┤ ░                               └───┘          »
q_6: ┤ Ry(θ[6]) ├┤ Rz(θ[14]) ├─░──────────────────────────────────────────────»
     ├──────────┤├───────────┤ ░                                              »
q_7: ┤ Ry(θ[7]) ├┤ Rz(θ[15]) ├─░──────────────────────────────────────────────»
     └──────────┘└───────────┘ ░                                              »
«                                                                           »
«q_0: ──■──────────────■────────────────────────────────────────────────────»
«       │              │                                                    »
«q_1: ──┼────■─────────┼────■──────────────■────────────────────────────────»
«       │    │         │    │              │                                »
«q_2: ──┼────┼────■────┼────┼────■─────────┼────■─────────■─────────────────»
«       │    │    │    │    │    │         │    │         │                 »
«q_3: ──┼────┼────┼────┼────┼────┼────■────┼────┼────■────┼────■─────────■──»
«       │    │  ┌─┴─┐  │    │    │  ┌─┴─┐  │    │    │    │    │         │  »
«q_4: ──┼────┼──┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼────┼────┼────■────┼──»
«       │  ┌─┴─┐└───┘  │    │  ┌─┴─┐└───┘  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │  »
«q_5: ──┼──┤ X ├───────┼────┼──┤ X ├───────┼────┼──┤ X ├──┼────┼──┤ X ├──┼──»
«     ┌─┴─┐└───┘       │  ┌─┴─┐└───┘       │  ┌─┴─┐└───┘  │  ┌─┴─┐└───┘  │  »
«q_6: ┤ X ├────────────┼──┤ X ├────────────┼──┤ X ├───────┼──┤ X ├───────┼──»
«     └───┘          ┌─┴─┐└───┘          ┌─┴─┐└───┘     ┌─┴─┐└───┘     ┌─┴─┐»
«q_7: ───────────────┤ X ├───────────────┤ X ├──────────┤ X ├──────────┤ X ├»
«                    └───┘               └───┘          └───┘          └───┘»
«                               ░ ┌───────────┐┌───────────┐ ░                »
«q_0: ──────────────────────────░─┤ Ry(θ[16]) ├┤ Rz(θ[24]) ├─░───■────■────■──»
«                               ░ ├───────────┤├───────────┤ ░ ┌─┴─┐  │    │  »
«q_1: ──────────────────────────░─┤ Ry(θ[17]) ├┤ Rz(θ[25]) ├─░─┤ X ├──┼────┼──»
«                               ░ ├───────────┤├───────────┤ ░ └───┘┌─┴─┐  │  »
«q_2: ──────────────────────────░─┤ Ry(θ[18]) ├┤ Rz(θ[26]) ├─░──────┤ X ├──┼──»
«                               ░ ├───────────┤├───────────┤ ░      └───┘┌─┴─┐»
«q_3: ──────────────────────────░─┤ Ry(θ[19]) ├┤ Rz(θ[27]) ├─░───────────┤ X ├»
«                               ░ ├───────────┤├───────────┤ ░           └───┘»
«q_4: ──■────■──────────────────░─┤ Ry(θ[20]) ├┤ Rz(θ[28]) ├─░────────────────»
«       │    │                  ░ ├───────────┤├───────────┤ ░                »
«q_5: ──┼────┼────■────■────────░─┤ Ry(θ[21]) ├┤ Rz(θ[29]) ├─░────────────────»
«     ┌─┴─┐  │  ┌─┴─┐  │        ░ ├───────────┤├───────────┤ ░                »
«q_6: ┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[22]) ├┤ Rz(θ[30]) ├─░────────────────»
«     └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤├───────────┤ ░                »
«q_7: ─────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[23]) ├┤ Rz(θ[31]) ├─░────────────────»
«          └───┘     └───┘└───┘ ░ └────

Finally, let' solve the problem using VQE! We have written code in the block below to solve the problem. Don't worry about the code - focus on the results you get. 

The results will be displayed as follows:

`result:`

`optimal function value: 17.0`

`optimal value: [0. 1. 1. 1.]`

`status: SUCCESS`

`solution:`

 `[1, 2, 3]`
 
Here, `optimal function value: 17.0` is the optimized value of your knapsack found by VQE. `optimal value: [0. 1. 1. 1.]` shows which items you should put in your knapsack to get this value - 0 means that you should omit this item, and 1 means you should keep it. In this example, you should keep the items with index 1, 2, and 3, and omit the item with index 0 (remember indices begin at 0). This successful solution is also displayed separately as `solution: [1, 2, 3]` . 

Can you verify that VQE solves the knapsack problem correctly?

In [9]:
# BLOCK 6 - Running VQE and printing results. 
calc = MinimumEigenOptimizer(method)
result = calc.solve(knapsack_problem.to_quadratic_program())
print('result:\n', result)
print('\nsolution:\n', knapsack_problem.interpret(result))

result:
 optimal function value: 17.0
optimal value: [0. 1. 1. 1.]
status: SUCCESS

solution:
 [1, 2, 3]
